# Introduction to LangChain.js

Welcome! This short course will introduce you to [LangChain.js](https://github.com/langchain-ai/langchainjs), a framework for building large language model (LLM) powered, context-aware, reasoning applications. By the end of the course, you'll learn all the concepts you need to create your own version of a popular example of this type of app: a "chat with data" chain that lets you ask questions about a document's contents using natural language.

This course follows on from the previous DeepLearning.ai courses on the Python version of LangChain. Much like LangChain.js itself, it's intended for web developers and others in the broader JavaScript ecosystem interested in building with LLMs, and will dive more deeply into features like streaming and integration with standard web/JavaScript APIs.

This notebook uses the [Deno](https://deno.com/) Jupyter kernel.

**Note:** Throughout this course, we'll link to some explorable traces in [LangSmith](https://smith.langchain.com/) that illustrate how the different example chains work. If you're following along locally, you'll need to set a few environment variables [as documented here](https://docs.smith.langchain.com/).

## Building Blocks: LLMs

Let's start with one of the most fundamental pieces of LangChain: the language model. LangChain includes two different types of language models: 

1. Text LLMs, which take a string as input and returns a string. An example of this is OpenAI's `text-davinci-003`, also known as GPT-3.
2. Chat Models, which take a list of messages as input and return a message. An example of this is OpenAI's `gpt-4`.

Because they are strings, LLM inputs and outputs are easy to visualize. So let's look at what calling a chat model directly looks like:

In [1]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [2]:
import { ChatOpenAI } from "langchain/chat_models/openai";
import { HumanMessage } from "langchain/schema";

const model = new ChatOpenAI({
  modelName: "gpt-3.5-turbo-1106",
});

await model.invoke([
  new HumanMessage("Tell me a joke."),
]);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "Why don't skeletons fight each other?\n\nThey don't have the guts!",
    additional_kwargs: { function_call: undefined, tool_calls: undefined }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "Why don't skeletons fight each other?\n\nThey don't have the guts!",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}

Above, we pass in an array with a single `HumanMessage` as input, and receive a single `AIMessage` as output. Messages have a `content` field containing the text value of the message, and an associated `role` that corresponds to the entity sending the message. 

**Note:** While this course will primarily use OpenAI's `gpt-3.5-turbo` chat model, LangChain supports models from many different providers, and you can try swapping the provided class in any of the code examples.

## Building Blocks: Prompt Templates

While calling models in isolation can be useful, it is often more convenient to factor out the logic behind model inputs into a reusable, parameterized component. For this purpose, LangChain includes prompt templates, which are responsible for formatting user input for later model calls. Input variables are denoted by curly braces within the template, and will be substituted into the final formatted value. 

Prompt templates are also useful for smoothing over some of the differences in model input types. Below, we construct a prompt template directly from a string:

In [3]:
import { ChatPromptTemplate } from "langchain/prompts";

const prompt = ChatPromptTemplate.fromTemplate(
  `What are three good names for a company that makes {product}?`
);

But we can use this prompt template to generate both string input for an LLM:

In [4]:
await prompt.format({
  product: "colorful socks",
});

"Human: What are three good names for a company that makes colorful socks?"

Or a message array for chat models:

In [5]:
await prompt.formatMessages({
  product: "colorful socks",
});

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What are three good names for a company that makes colorful socks?",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What are three good names for a company that makes colorful socks?",
    name: undefined,
    additional_kwargs: {}
  }
]

You'll notice that if we use the `.fromTemplate()` method, for convenience the input string gets wrapped in a `HumanMessage` and formatted as an array, which matches the input our chat model expects.

We can also create a prompt template from messages directly for finer-grained control over what types of messages are passed to the prompt:

In [6]:
import { 
  SystemMessagePromptTemplate, 
  HumanMessagePromptTemplate 
} from "langchain/prompts";

const promptFromMessages = ChatPromptTemplate.fromMessages([
  SystemMessagePromptTemplate.fromTemplate(
    "You are an expert at picking company names."
  ),
  HumanMessagePromptTemplate.fromTemplate(
    "What are three good names for a company that makes {product}?"
  )
]);

await promptFromMessages.formatMessages({
  product: "shiny objects"
});

[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "You are an expert at picking company names.",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "You are an expert at picking company names.",
    name: undefined,
    additional_kwargs: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What are three good names for a company that makes shiny objects?",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What are three good names for a company that makes shiny objects?",
    name: undefined,
    additional_kwargs: {}
  }
]

The system message helps set the behavior of the assistant for many models, including OpenAI.

Or for short, you can use a tuple with the role and the template:

In [7]:
const promptFromMessages = ChatPromptTemplate.fromMessages([
  ["system", "You are an expert at picking company names."],
  ["human", "What are three good names for a company that makes {product}?"]
]);

await promptFromMessages.formatMessages({
  product: "shiny objects"
});

[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "You are an expert at picking company names.",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "You are an expert at picking company names.",
    name: undefined,
    additional_kwargs: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What are three good names for a company that makes shiny objects?",
      additional_kwargs: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What are three good names for a company that makes shiny objects?",
    name: undefined,
    additional_kwargs: {}
  }
]

Though we can pass these formatted values directly into a model, there's a more elegant way to use prompts and models together that we'll go over next.

## Building Blocks: LangChain Expression Language

LangChain Expression Language (LCEL) is a composable syntax for chaining LangChain modules together. Objects that are compatible with LCEL are called `Runnables`.

We can construct a simple chain from the prompt and model we declared above like this:

In [8]:
const chain = prompt.pipe(model);

The input of the chain is the same as the first step in the sequence, which in this case is an object with a single `product` property. The prompt template is invoked with this input, then passes the properly formatted result as input into the next step of the chain, the chat model. Here's what it looks like in action:

In [9]:
await chain.invoke({
  product: "colorful socks",
});

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "1. ChromaSock\n2. RainbowFootwear\n3. SpectrumSocks",
    additional_kwargs: { function_call: undefined, tool_calls: undefined }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "1. ChromaSock\n2. RainbowFootwear\n3. SpectrumSocks",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined }
}

We can see that the result is an `AIMessage` with three good names for a company that makes colorful socks.

## Building Blocks: Output Parser

The final consideration we'll go over in this section is formatting our output. For example, it is often easier to work with the raw string value of a chat model's output rather than an AI message. The LangChain abstraction for this is called an output parser. 

Below, we redefine our chain with an output parser that coerces the message output from the chat model into a string as a third and final step:

In [10]:
import { StringOutputParser } from "langchain/schema/output_parser";

const outputParser = new StringOutputParser();

const nameGenerationChain = prompt.pipe(model).pipe(outputParser);

And now, if we invoke the chain, we can see that the output is a raw string:

In [11]:
await nameGenerationChain.invoke({
  product: "fancy cookies",
});

"1. Gourmet Crumb\n2. Delightful Desserts\n3. Elegant Eats"

You can see a LangSmith trace of the above example [here](https://smith.langchain.com/public/9fd7689f-23c6-4ea1-8a76-fea1b5f8d5db/r).

These three pieces form the core of many more complicated chains.

## Streaming

One of the many advantages to using LCEL is that chains get certain methods by default. One useful one is `.stream()`, which returns output from the chain in an iterable stream. Because LLM responses often take a long time to finish, streaming is useful in situations where showing feedback quickly is important.

Here's an example with the chain we just composed:

In [12]:
const stream = await nameGenerationChain.stream({
  product: "really cool robots",
});

for await (const chunk of stream) {
  console.log(chunk);
}


1
.
 Rob
o
Tech
 Innov
ations


2
.
 Cy
borg
 Cre
ations


3
.
 Fut
ura
 Robotics
 Co
.



Above, the `model` emits partial message chunks, and the output parser transforms those streamed chunks from the model as they are generated, resulting in string output chunks.

## Batch

LCEL also gives us `.batch()` out of the box too for easier concurrent operations:

In [13]:
const inputs = [{
  product: "large calculators"
}, {
  product: "alpaca wool sweaters"
}]

await nameGenerationChain.batch(inputs);

[
  "1. MegaCalc Co.\n2. TitanTech Calculators\n3. JumboCalc Solutions",
  "1. Alpaca Threads\n2. Andean Knits\n3. Luxe Alpaca Wear"
]